<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f0133609d2af646ed20c857e6980a5b07fb4616ce9b8ee0a424e97c95ece1041
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-06 11:03:13
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: -1.13 L (-0.78%)
Current PnL: -21.16 L (-13.87%)
CY Booked + Current PnL: -8.51 L (-5.58%)
-------------------
Total profit:  1.99 L
Total loss:  -23.16 L
-------------------
Total Booked + Current PnL: 18.47 L (14.7%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.82%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.15 L (61.47%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-8.03,62.0,X-LC,5.56,229804.0,19438.0,9353.0,-0.78,9.24,4.07,13.68,37.0,2.08,1.61,27.29,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-5.43,67.0,M-LC,10.69,172346.0,31506.0,12978.0,0.03,22.37,7.53,31.59,66.0,2.43,1.21,54.78,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,103.84,54.0,M-SC,1.77,87809.0,-12968.0,13057.0,0.22,-12.87,14.87,0.09,245.0,-0.99,0.62,15.36,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.65,40.0,X-MC,2.61,88185.0,-2493.0,17399.0,0.12,-2.75,19.73,16.44,101.0,-0.14,0.62,16.87,X40,ATH,PHARMA
33,ICICIGI,2252.93,-13.84,57.0,X-MC,5.54,148222.0,12089.0,18498.0,-1.81,8.88,12.48,22.47,91.0,0.65,1.04,23.01,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,-9.40,69.0,X-LC,4.63,228694.0,-23074.0,76292.0,4.51,-9.16,33.36,21.13,27.0,-0.30,1.61,23.01,X40,ATH,PAINTS
24,EASEMYTRIP,26.40,-16.61,55.0,M-SC,27.30,80211.0,-96604.0,177403.0,3.66,-54.64,221.17,45.70,197.0,-0.54,0.56,4.71,XY24,NTT,TRAVEL
13,BERGEPAINT,680.00,-17.10,59.0,X-MC,1.32,222790.0,-4549.0,52601.0,2.35,-2.00,23.61,21.14,106.0,-0.09,1.57,25.58,XY24,NTT,PAINTS
67,SIS,528.00,2018.65,45.0,H-SC,2.74,85502.0,-25530.0,49138.0,1.30,-22.99,57.47,21.26,156.0,-0.52,0.60,15.52,OX40N,NTT,MISC
21,DABUR,735.00,-6.18,60.0,X-MC,2.91,253849.0,5781.0,102631.0,1.23,2.33,40.43,43.70,102.0,0.06,1.78,17.11,XY24,BTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SFL,1287.0,33.53,49.0,M-SC,4.31,176188.0,-86030.0,155856.0,-4.79,-32.81,88.46,26.63,229.0,-0.55,1.24,9.54,XY24,NTT,MISC
8,ATULAUTO,844.0,3375.68,35.0,M-SC,3.29,156060.0,-41072.0,130903.0,-3.94,-20.83,83.88,45.57,236.0,-0.31,1.10,10.64,XY24,NTT,AUTO
71,SYMPHONY,1306.0,-44.18,47.0,M-SC,17.02,119151.0,-51990.0,51938.0,-3.01,-30.38,43.59,-0.03,196.0,-1.00,0.84,3.88,OX40N,NTT,DURABLES
65,SHALBY,327.0,1102.55,42.0,M-SC,18.48,163583.0,-17560.0,63029.0,-2.77,-9.69,38.53,25.10,235.0,-0.28,1.15,31.10,XY24,NTT,HEALTHCARE
74,TATAMOTORS,1065.0,-53.17,21.0,X-LC,28.48,147560.0,-128963.0,239032.0,-2.52,-46.64,161.99,39.81,54.0,-0.54,1.04,4.00,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,42.86,45.0,M-LC,4.99,201438.0,-1732.0,43853.0,-1.19,-0.85,21.77,20.74,55.0,-0.04,1.42,5.95,XY25,NTT,FINANCE
31,HINDZINC,730.22,24.68,43.0,M-LC,10.43,203232.0,-1844.0,114420.0,-1.30,-0.90,56.30,54.89,52.0,-0.02,1.43,22.98,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,103.84,54.0,M-SC,1.77,87809.0,-12968.0,13057.0,0.22,-12.87,14.87,0.09,245.0,-0.99,0.62,15.36,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,44.0,H-SC,2.67,221427.0,-48240.0,84563.0,-1.01,-17.89,38.19,13.47,138.0,-0.57,1.56,12.59,XY24,NTT,PAINTS
17,CERA,9475.0,-22.20,51.0,H-SC,1.78,141657.0,-34246.0,76268.0,-2.15,-19.47,53.84,23.89,149.0,-0.45,1.00,22.34,OX40N,NTT,CERAMICS
67,SIS,528.0,2018.65,45.0,H-SC,2.74,85502.0,-25530.0,49138.0,1.30,-22.99,57.47,21.26,156.0,-0.52,0.60,15.52,OX40N,NTT,MISC
42,JCHAC,2282.0,19921.74,63.0,M-SC,1.52,97888.0,-29917.0,29905.0,0.03,-23.41,30.55,-0.01,232.0,-1.00,0.69,10.84,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,67.0,M-LC,10.69,172346.0,31506.0,12978.0,0.03,22.37,7.53,31.59,66.0,2.43,1.21,54.78,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,49.96,59.0,M-MC,9.94,236465.0,11503.0,156161.0,-0.62,5.11,66.04,74.53,192.0,0.07,1.66,39.02,XY24,BTT,STEEL
46,KPIGREEN,731.05,9.79,61.0,H-SC,7.45,130700.0,5403.0,53522.0,-2.52,4.31,40.95,47.03,141.0,0.10,0.92,63.65,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,67.0,M-LC,10.69,172346.0,31506.0,12978.0,0.03,22.37,7.53,31.59,66.0,2.43,1.21,54.78,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-39.53,56.0,M-SC,5.40,125286.0,10488.0,83553.0,-0.92,9.14,66.69,81.92,223.0,0.13,0.88,48.32,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.84,129945.0,6609.0,125020.0,-0.66,5.36,96.21,106.72,119.0,0.05,0.91,29.99,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-39.53,56.0,M-SC,5.40,125286.0,10488.0,83553.0,-0.92,9.14,66.69,81.92,223.0,0.13,0.88,48.32,XR,NTT,DURABLES
46,KPIGREEN,731.05,9.79,61.0,H-SC,7.45,130700.0,5403.0,53522.0,-2.52,4.31,40.95,47.03,141.0,0.10,0.92,63.65,MH,ATH,POWER
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.84,129945.0,6609.0,125020.0,-0.66,5.36,96.21,106.72,119.0,0.05,0.91,29.99,AR,ATH,MISC
86,WIPRO,420.00,-15.61,41.0,M-LC,5.50,148261.0,-2684.0,112145.0,0.51,-1.78,75.64,72.51,53.0,-0.02,1.04,4.14,XR,NTT,IT
31,HINDZINC,730.22,24.68,43.0,M-LC,10.43,203232.0,-1844.0,114420.0,-1.30,-0.90,56.30,54.89,52.0,-0.02,1.43,22.98,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,21.0,X-LC,28.48,147560.0,-128963.0,239032.0,-2.52,-46.64,161.99,39.81,54.0,-0.54,1.04,4.00,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.72,25.0,X-SC,6.95,87233.0,-41437.0,86736.0,-1.42,-32.20,99.43,35.20,219.0,-0.48,0.61,0.12,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-23.55,28.0,X-MC,4.70,192105.0,-8242.0,38748.0,-1.76,-4.11,20.17,15.23,178.0,-0.21,1.35,26.81,X40N,NTT,REALTY
23,DMART,5391.45,-19.03,32.0,X-LC,8.30,206255.0,-4152.0,63320.0,-1.37,-1.97,30.70,28.12,38.0,-0.07,1.45,21.19,X40N,ATH,FMCG
53,QUESS,986.00,-50.41,33.0,X-SC,37.75,49231.0,-15775.0,166701.0,-2.20,-24.27,338.61,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.21,36.0,X-MC,0.91,199110.0,-1710.0,69509.0,-0.95,-0.85,34.91,33.76,80.0,-0.02,1.40,4.30,X40,ATH,FMCG
41,ITC,452.00,-37.92,47.0,X-LC,1.12,199903.0,-235.0,21130.0,-0.02,-0.12,10.57,10.44,4.0,-0.01,1.40,5.44,X40,NTT,FMCG
66,SIEMENS,4671.50,-3.25,45.0,H-LC,1.26,155350.0,-30745.0,78219.0,-0.40,-16.52,50.35,25.51,15.0,-0.39,1.09,14.48,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-7.74,39.0,X-MC,3.03,307755.0,-20236.0,126764.0,-1.41,-6.17,41.19,32.48,92.0,-0.16,2.16,4.55,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.38,65.0,X-LC,4.11,245341.0,11197.0,37439.0,-1.69,4.78,15.26,20.77,86.0,0.30,1.72,12.03,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,33.0,X-SC,37.75,49231.0,-15775.0,166701.0,-2.20,-24.27,338.61,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.72,25.0,X-SC,6.95,87233.0,-41437.0,86736.0,-1.42,-32.20,99.43,35.20,219.0,-0.48,0.61,0.12,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-29.24,34.0,X-MC,7.23,79900.0,-3610.0,23642.0,-0.46,-4.32,29.59,23.98,82.0,-0.15,0.56,1.25,X40,ATH,APPARELS
20,COLPAL,3726.84,-6.96,40.0,X-MC,7.34,217390.0,-45975.0,155303.0,-0.77,-17.46,71.44,41.51,84.0,-0.30,1.53,2.02,XY25,ATH,FMCG
3,ACC,3906.00,-38.50,44.0,X-MC,2.44,183300.0,-54651.0,207294.0,0.02,-22.97,113.09,64.15,174.0,-0.26,1.29,2.82,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,33.0,X-SC,37.75,49231.0,-15775.0,166701.0,-2.20,-24.27,338.61,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.24,34.0,X-MC,7.23,79900.0,-3610.0,23642.0,-0.46,-4.32,29.59,23.98,82.0,-0.15,0.56,1.25,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.85,39.0,X-SC,4.33,80652.0,-64508.0,140439.0,-0.97,-44.44,174.13,52.31,136.0,-0.46,0.57,7.96,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.72,25.0,X-SC,6.95,87233.0,-41437.0,86736.0,-1.42,-32.20,99.43,35.20,219.0,-0.48,0.61,0.12,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.65,40.0,X-MC,2.61,88185.0,-2493.0,17399.0,0.12,-2.75,19.73,16.44,101.0,-0.14,0.62,16.87,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.01,48.0,X-LC,9.88,283062.0,-62894.0,129586.0,0.71,-18.18,45.78,19.28,1.0,-0.49,1.99,4.64,X40,ATH,IT
39,INFY,2275.00,-19.60,50.0,X-LC,5.12,313898.0,1010.0,170666.0,0.40,0.32,54.37,54.87,3.0,0.01,2.21,8.63,X40,BTT,IT
41,ITC,452.00,-37.92,47.0,X-LC,1.12,199903.0,-235.0,21130.0,-0.02,-0.12,10.57,10.44,4.0,-0.01,1.40,5.44,X40,NTT,FMCG
83,VBL,671.64,-16.13,55.0,X-LC,6.86,301838.0,-14004.0,126681.0,0.45,-4.43,41.97,35.67,5.0,-0.11,2.12,8.81,X40N,ATH,FMCG
1,ABB,7934.00,-39.48,54.0,H-LC,5.08,252840.0,-8779.0,127988.0,0.45,-3.36,50.62,45.57,7.0,-0.07,1.78,7.59,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,37.0,L-SC,27.77,75802.0,-37747.0,77796.0,-2.45,-33.24,102.63,35.27,268.0,-0.49,0.53,85.53,XR,NTT,HOTELS
7,ASIANTILES,137.00,7494.44,70.0,L-SC,10.68,83348.0,-10462.0,87082.0,-0.22,-11.15,104.48,81.67,269.0,-0.12,0.59,61.41,XR,NTT,CERAMICS
49,MASFIN,398.61,-18.70,50.0,H-SC,8.37,92325.0,-5655.0,27254.0,-1.61,-5.77,29.52,22.05,152.0,-0.21,0.65,34.20,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-39.53,56.0,M-SC,5.40,125286.0,10488.0,83553.0,-0.92,9.14,66.69,81.92,223.0,0.13,0.88,48.32,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.84,129945.0,6609.0,125020.0,-0.66,5.36,96.21,106.72,119.0,0.05,0.91,29.99,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-52.31,68.0,H-SC,7.33,271395.0,-15110.0,272453.0,-2.45,-5.27,100.39,89.82,134.0,-0.06,1.91,43.59,XY24,NTT,FINANCE
81,VAIBHAVGBL,521.00,74.10,64.0,H-SC,2.30,151039.0,-31736.0,142807.0,-2.46,-17.36,94.55,60.77,125.0,-0.22,1.06,35.28,XR,NTT,JEWELLERY
46,KPIGREEN,731.05,9.79,61.0,H-SC,7.45,130700.0,5403.0,53522.0,-2.52,4.31,40.95,47.03,141.0,0.10,0.92,63.65,MH,ATH,POWER
68,SONACOMS,806.63,-32.06,66.0,M-SC,6.91,84612.0,-16547.0,56546.0,-0.06,-16.36,66.83,39.54,202.0,-0.29,0.59,19.31,AR,ATH,AUTO
7,ASIANTILES,137.00,7494.44,70.0,L-SC,10.68,83348.0,-10462.0,87082.0,-0.22,-11.15,104.48,81.67,269.0,-0.12,0.59,61.41,XR,NTT,CERAMICS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.76
1,25,44.41
2,50,75.56


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.75
LC    33.19
MC    23.08
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.02
X40      16.24
X40N     12.42
XY25     11.94
XR        9.84
AR        7.85
OX40N     6.53
X200      1.78
X5K       1.43
SR        1.05
MH        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.23
X-LC    22.46
X-MC    18.78
M-SC    12.59
X-SC     5.48
M-LC     5.10
H-LC     4.65
H-MC     2.33
M-MC     1.66
L-SC     1.45
L-LC     0.98
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.33,-3.71,37.24
IT,12.42,-18.96,82.70
FINANCE,12.19,-9.01,58.42
MISC,7.69,-18.63,78.46
ELECTRICAL,5.79,-9.60,48.45
PAINTS,5.60,-16.01,33.30
BANKS,4.03,-17.50,85.85
INSURANCE,3.84,-1.32,39.10
AUTO,2.73,-48.06,109.85


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3303462.0,22
XR,1253604.0,13
AR,1170346.0,9
X40,845747.0,12
X40N,701010.0,9
OX40N,555572.0,9
XY25,474388.0,8
SR,271987.0,2
X5K,114420.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3381107.0,24
M-SC,1253315.0,15
X-MC,1202182.0,14
X-LC,1082901.0,13
X-SC,569334.0,6
M-LC,283396.0,4
H-LC,276745.0,3
H-MC,256403.0,2
L-SC,254817.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1184491.0      6
           AR         840024.0      5
           XR         786268.0      7
M-SC       XY24       723933.0      6
X-MC       XY24       564121.0      4
X-LC       X40        460814.0      5
X-SC       X40N       315897.0      4
X-LC       XY24       312988.0      2
X-MC       X40        298197.0      6
H-SC       SR         271987.0      2
           OX40N      244815.0      3
X-LC       X40N       227440.0      3
H-LC       AR         206207.0      2
X-MC       XY25       182191.0      2
H-MC       XY24       182089.0      1
X-SC       XY24       166701.0      1
L-SC       XR         164878.0      2
X-MC       X40N       157673.0      2
M-MC       XY24       156161.0      1
M-SC       OX40N      146504.0      4
           XR         132384.0      2
           XY25       126379.0      1
           AR         124115.0      2
M-LC       X5K        114420.0      1
           XR         112145.0      1
L-SC       OX40N       89939.0      1
X-SC       X40         86736.0      1
X-LC       XY25        81659.0      3
H-MC       OX40N       74314.0      1
H-LC       X200        70538.0      1
L-MC       XR          57929.0      1
H-SC       MH          53522.0      1
M-LC       XY25        43853.0      1
L-LC       XY25        40306.0      1
M-LC       XY24        12978.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
